# StyleCLIP+: Towards an Extended and User-Friendly StyleCLIP, <b>Gradio Demo 3</b>
Fall 2023 CS470 Introduction to Artificial Intelligence Project 2 (Implementation), Team 3  
Myeongseok Kwon<sup>1</sup>, Junhak Ha<sup>2</sup>, Dongwan Hong<sup>3</sup>, Kyeongmin Lee<sup>4</sup>, Bryan Nathanael Wijaya<sup>5*</sup>  
<small><sup>1</sup>20170042, <sup>2</sup>20190683, <sup>3</sup>20190696, <sup>4</sup>20200429, <sup>5</sup>20200735, <sup>*</sup>Team Leader</small>

## Simplified Global Direction

In [1]:
#@title Setting up Gradio for demo
!pip install kaleido cohere openai tiktoken gradio -q
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.8 MB/s

In [2]:
#@title Environment setup

import os
# General environment setup for global directions
os.chdir('/content')
CODE_DIR = 'encoder4editing'

! pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

!git clone https://github.com/omertov/encoder4editing.git $CODE_DIR
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

%cd
%cd /content/

!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!git clone https://github.com/bramyeon/StyleCLIP

os.chdir(f'./{CODE_DIR}')

from argparse import Namespace
import time
import os
import sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from utils.common import tensor2im
from models.psp import pSp
%load_ext autoreload
%autoreload 2
%matplotlib inline

%cd
%cd /content/StyleCLIP/global_torch/

dataset_name = 'ffhq'
if not os.path.isfile('./model/'+dataset_name+'.pkl'):
        url='https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/'
        name='stylegan2-'+dataset_name+'-config-f.pkl'
        os.system('wget ' +url+name + '  -P  ./model/')
        os.system('mv ./model/'+name+' ./model/'+dataset_name+'.pkl')
# input prepare data
import clip
from manipulate import Manipulator
from StyleCLIP import GetDt,GetBoundary
from gdown import download as drive_download

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device,jit=False)

network_pkl='./model/'+dataset_name+'.pkl'
device = torch.device('cuda')
M=Manipulator()
M.device=device
G=M.LoadModel(network_pkl,device)
M.G=G
M.SetGParameters()
num_img=100_000
M.GenerateS(num_img=num_img)
M.GetCodeMS()
np.set_printoptions(suppress=True)

file_path='./npy/'+dataset_name+'/'
fs3=np.load(file_path+'fs3.npy')

drive_download("https://drive.google.com/uc?id=1O8OLrVNOItOJoNGMyQ8G8YRTeTYEfs0P", "/content/encoder4editing/e4e_ffhq_encode.pt", quiet=False)

os.chdir('/content/encoder4editing')

EXPERIMENT_ARGS = {
        "model_path": "e4e_ffhq_encode.pt"
    }
EXPERIMENT_ARGS['transform'] = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
resize_dims = (256, 256)

model_path = EXPERIMENT_ARGS['model_path']
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
# pprint.pprint(opts)  # Display full options used
# update the training options
opts['checkpoint_path'] = model_path
opts= Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
from PIL import Image

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu110 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 166MiB/s]


name:conv1_resolution_4 Resolution: 4, InC: 512, OutC:512, w_dim: 512
name:toRGB_resolution_4 InC: 512, OutC:3, w_dim: 512
name:conv0_resolution_8 Resolution: 8, InC: 512, OutC:512, w_dim: 512
name:conv1_resolution_8 Resolution: 8, InC: 512, OutC:512, w_dim: 512
name:toRGB_resolution_8 InC: 512, OutC:3, w_dim: 512
name:conv0_resolution_16 Resolution: 16, InC: 512, OutC:512, w_dim: 512
name:conv1_resolution_16 Resolution: 16, InC: 512, OutC:512, w_dim: 512
name:toRGB_resolution_16 InC: 512, OutC:3, w_dim: 512
name:conv0_resolution_32 Resolution: 32, InC: 512, OutC:512, w_dim: 512
name:conv1_resolution_32 Resolution: 32, InC: 512, OutC:512, w_dim: 512
name:toRGB_resolution_32 InC: 512, OutC:3, w_dim: 512
name:conv0_resolution_64 Resolution: 64, InC: 512, OutC:512, w_dim: 512
name:conv1_resolution_64 Resolution: 64, InC: 512, OutC:512, w_dim: 512
name:toRGB_resolution_64 InC: 512, OutC:3, w_dim: 512
name:conv0_resolution_128 Resolution: 128, InC: 512, OutC:256, w_dim: 512
name:conv1_resol

Downloading...
From: https://drive.google.com/uc?id=1O8OLrVNOItOJoNGMyQ8G8YRTeTYEfs0P
To: /content/encoder4editing/e4e_ffhq_encode.pt
100%|██████████| 1.20G/1.20G [00:15<00:00, 76.3MB/s]


Loading e4e over the pSp framework from checkpoint: e4e_ffhq_encode.pt


In [4]:
#@title Demo 3 Gradio

def demo3_fn(dataset_name, original_image, target, neutral, alpha, beta):
    # Align image
    if dataset_name == 'ffhq':
        experiment_type = 'ffhq_encode'
    else:
        experiment_type = 'others'

    original_image = Image.fromarray(original_image)
    original_image = original_image.convert('RGB')
    image_path = 'original_image.jpg'
    original_image.save(image_path)

    if experiment_type == "ffhq_encode" and 'shape_predictor_68_face_landmarks.dat' not in os.listdir():
        !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
        !bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

    def run_alignment(image_path):
        import dlib
        from utils.alignment import align_face
        predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
        aligned_image = align_face(filepath=image_path, predictor=predictor)
        return aligned_image

    if experiment_type == "ffhq_encode":
        input_image = run_alignment(image_path)
    else:
        input_image = original_image

    input_image.resize(resize_dims)

    # Invert the image
    img_transforms = EXPERIMENT_ARGS['transform']
    transformed_image = img_transforms(input_image)

    def display_alongside_source_image(result_image, source_image):
        res = np.concatenate([np.array(source_image.resize(resize_dims)),
                            np.array(result_image.resize(resize_dims))], axis=1)
        return Image.fromarray(res)

    def run_on_batch(inputs, net):
        images, latents = net(inputs.to("cuda").float(), randomize_noise=False, return_latents=True)
        if experiment_type == 'cars_encode':
            images = images[:, :, 32:224, :]
        return images, latents

    with torch.no_grad():
        images, latents = run_on_batch(transformed_image.unsqueeze(0), net)
        result_image, latent = images[0], latents[0]
    torch.save(latents, 'latents.pt')

    mode='real image'

    if mode == 'real image':
        img_index = 0
        latents=torch.load('/content/encoder4editing/latents.pt')
        dlatents_loaded=M.G.synthesis.W2S(latents)

        img_indexs=[img_index]
        dlatents_loaded=M.S2List(dlatents_loaded)

        dlatent_tmp=[tmp[img_indexs] for tmp in dlatents_loaded]
    elif mode == 'generated image':
        img_indexs=[img_index]
        dlatents_loaded=M.S2List(dlatents_loaded)
        dlatent_tmp=[tmp[img_indexs] for tmp in M.dlatents]
    M.num_images=len(img_indexs)

    M.alpha=[0]
    M.manipulate_layers=[0]
    codes,out=M.EditOneC(0,dlatent_tmp)
    original=Image.fromarray(out[0,0]).resize((512,512))
    M.manipulate_layers=None

    classnames=[target,neutral]
    dt=GetDt(classnames,model)

    # make the original and generated images up-and-down
    def display(original, generated):
        min_width = min(original.width, generated.width)
        original = original.resize((min_width, int(original.height * min_width / original.width)))
        generated = generated.resize((min_width, int(generated.height * min_width / generated.width)))
        new_width = min_width
        new_height = original.height + generated.height
        new_image = Image.new('RGB', (new_width, new_height))
        new_image.paste(original, (0, 0))
        new_image.paste(generated, (0, original.height))
        return new_image

    M.alpha=[alpha]
    boundary_tmp2,c=GetBoundary(fs3,dt,M,threshold=beta)
    codes=M.MSCode(dlatent_tmp,boundary_tmp2)
    out=M.GenerateImg(codes)
    generated=Image.fromarray(out[0,0])#.resize((512,512))
    result = display(original, generated)

    return result

demo3 = gr.Interface(
    fn=demo3_fn, title="CS470 Team 3 Global Direction Demo",
    inputs=[gr.Dropdown(choices=['ffhq'], value='ffhq', label='Image domain (ffhq: face)'), \
            gr.Image(label="Original image"), \
            gr.Textbox(lines=1, placeholder="e.g. a face with blue eyes", label="Target prompt"), \
            gr.Textbox(lines=1, placeholder="e.g. a face with eyes", label="Neutral prompt"), \
            gr.Slider(-10, 10, value=4, step=0.1, label='Manipulation strength (alpha)'), \
            gr.Slider(0.08, 0.3, value=0.15, step=0.01,label='Features intertwining degree (beta)')],
    outputs=[gr.Image(label="Original vs. target image")])

if __name__ == "__main__":
    demo3.launch(show_api=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://da26e6e9fa5e010473.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
